<a id="setup"></a>
## LangChain memory with watsonx.ai models

This notebook contains sample code for using memory buffers with models included in watsonx.ai.


### Install dependecies

In [ ]:
import sys

!{sys.executable} -m pip install ibm-watsonx-ai | tail -n 1
!{sys.executable} -m pip install langchain | tail -n 1
!{sys.executable} -m pip install langchain-community | tail -n 1
!{sys.executable} -m pip install -U langchain-ibm | tail -n 1

### Define the WML credentials

This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [2]:
import getpass
from os import environ

try:
    REGION = environ["RUNTIME_ENV_REGION"]
except KeyError:
    # Set your region here if you are not running this notebook in the watsonx.ai Jupyter environment
    # us-south, eu-de, etc.
    REGION = "us-south"

credentials = {
    "url": "https://" + REGION + ".ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): "),
}

### Define the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [3]:
from os import getenv

try:
    project_id = environ["PROJECT_ID"]
except KeyError:
    # Enter project ID here if not running this notebook in the watsonx.ai Jupyter environment
    project_id = "MY_PROJECT_ID"

### Import required packages

In [4]:
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

In [ ]:
print("Supported Models:")

for model in ModelTypes:
    print("-", model.name)

### Create the LangChain model object

In [ ]:
# We will use the flan model

model_parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 300,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1,
}

# Experiment with different models. We noticed that Flan produces a more concise ouput, but llama - more descriptive and better quality
current_model = Model(
    model_id=ModelTypes.LLAMA_3_70B_INSTRUCT,
    params=model_parameters,
    credentials=credentials,
    project_id=project_id,
)

# Create the model objects that will be used by LangChain
current_llm = current_model.to_langchain()

### Create conversation

Use [`RunnableWithMessageHistory`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) (replacement of `ConversationChain`) to create a runnable that manages chat message history for another Runnable. See [migrating from ConversationalChain](https://python.langchain.com/docs/versions/migrating_chains/conversation_chain/) for more information.

In [18]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
local_memory = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in local_memory:
        local_memory[session_id] = ChatMessageHistory()
    return local_memory[session_id]

In [26]:
from langchain import PromptTemplate

base_prompt = PromptTemplate.from_template(
    template="You are virtual assistant working in customer service. Perform the following task to the best of your ability: {task}"
)

base_chain = base_prompt | current_llm

In [33]:
from langchain_core.runnables.history import RunnableWithMessageHistory

conversation = RunnableWithMessageHistory(
    base_chain,
    get_session_history=get_by_session_id,  # Function that returns a new BaseChatMessageHistory.
    input_messages_key="task",  # key in the input dict that contains the messages.
    history_messages_key="history",  # Use separate key for historical messages.
)

In [34]:
# Example of a first quesiton from the user
user_input = "From the following customer complaint, extract 3 factors that caused the customer to be unhappy. \
                            Put each factor on a new line. Customer complaint: I am writing you this statement to delete the \
                            following information on my credit report. The items I need deleted are listed in the report. \
                            I am a victim of identity thief, I demand that you remove these errors to correct my report immediately! \
                            I have reported this to the federal trade commission and have attached the federal trade commission affidavit. \
                            Now that I have given you the following information, please correct my credit report or I shall proceed with involving my attorney! \
                            Numbered list of complaints:"

# user_input="Right now I am bothered! I have attempted to be patient however it is hard to be patient when you feel that you are continually being overlooked by somebody. I think you fail to remember that \Consumer detailing organizations have expected an essential part in amassing and assessing customer credit and other data on shoppers. The XXXX XXXX  is reliant on the reasonable and precision."

Invoke the LLM chain using `user_input` as input.

> *Pro tip:* see [`RunnableConfig`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.config.RunnableConfig.html#langchain_core.runnables.config.RunnableConfig) for more information on how to invoke these flows.

In [ ]:
conversation.invoke(
    input={"task": user_input}, config={"configurable": {"session_id": "foo"}}
)

Now, invoke it for a different input, as a follow-up question:

In [ ]:
# Example of a second quesiton from the user
user_input = "Does the numbered list of complaints contain a statement about identity fraud? Provide a short answer: yes or no."

conversation.invoke(
    input={"task": user_input}, config={"configurable": {"session_id": "foo"}}
)

You can print the memory object to see the conversation history:

In [ ]:
# For debugging, print the history of the conversation
print(local_memory)

### Authors: 
- **Elena Lowery**, Data and AI Architect
- **Josefina Casanova**, Engagement Lead, Build Lab Americas. Edited for L4 watsonx course. 2024